In [2]:
from pynhd import NLDI, WaterData
import fsspec
import boto3
import geopandas as gpd
import math

In [2]:
# bbox (west, south, east, north)
%time
#huc12_basins = WaterData('wbd12').bybox((-124.7844079, 24.7433195, -66.9513812, 49.3457868), box_crs=4326, sort_attr='huc12')
huc12_basins = WaterData('wbd12').bybox((-124.7844079, 24.7433195, -66.9513812, 49.3457868), box_crs=4326)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs


ServiceError: Service returned the following error message:
URL: https://labs.waterdata.usgs.gov/geoserver/wmadata/ows?service=wfs&version=2.0.0&outputFormat=application/json&request=GetFeature&typeName=wmadata:wbd12_20201006&bbox=-124.7844079,24.7433195,-66.9513812,49.3457868,EPSG:4326&srsName=EPSG:4269&startIndex=22000&count=1000&sortBy=objectid
ERROR: <html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
</body>
</html>



In [ ]:
huc12_basins.plot()

In [ ]:
huc12_basins.head()

In [ ]:
fs_write = fsspec.filesystem(
    's3',
    profile='osn-renc',  ## This is the profile name you configured above.
    client_kwargs={'endpoint_url': 'https://renc.osn.xsede.org'}
)

# Write Geoparquet

In [ ]:
fname='rsignellbucket2/nwc/huc12/huc12.geoparquet'
%time
with fs_write.open(fname, mode='wb') as f:
    huc12_basins.to_parquet(f)

In [ ]:
fs_write.ls('rsignellbucket2/nwc/huc12/')

In [ ]:
def convert_size(size_bytes):
   if size_bytes == 0:
       return "0B"
   size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
   i = int(math.floor(math.log(size_bytes, 1024)))
   p = math.pow(1024, i)
   s = round(size_bytes / p, 2)
   return "%s %s" % (s, size_name[i])

In [ ]:
print(f'size of huc12 geoparquet: {convert_size(fs_write.size(fname))}')

In [ ]:
with fs_write.open(fname, mode='rb') as f:
    huc12_df = gpd.read_parquet(f)

In [ ]:
huc12_df.head()

In [ ]:
huc12_df.plot()

# Write geopackagefname='rsignellbucket2/nwc/huc12/huc12.gpkg'


In [ ]:
fname='rsignellbucket2/nwc/huc12/huc12.gpkg'

In [ ]:
import boto3
session = boto3.Session(profile_name='osn-renc')
#huc12_basins.to_file(fname, layer='huc12', driver="GPKG")

In [ ]:
%time
with fs_write.open(fname, mode='wb') as f:
    huc12_basins.to_file(f, layer='huc12', driver="GPKG")

In [ ]:
fs_write.ls('rsignellbucket2/nwc/huc12/')

In [ ]:
print(f'size of huc12 geopackage: {convert_size(fs_write.size(fname))}')

In [ ]:
%time
with fs_write.open(fname, mode='rb') as f:
    huc12_df = gpd.read_file(f, layer='huc12', driver="GPKG")  

In [ ]:
huc12_df.head()

In [ ]:
huc12_df.plot()